![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_XLNet.ipynb)

## Import XLNet models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only available in `Spark NLP 3.1.2` and above. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for XLNet from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use XLNet models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLNetTokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
pandas

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [xlnet-base-cased](https://huggingface.co/xlnet-base-cased) model from HuggingFace as an example
- In addition to `TFXLNetModel` we also need to save the `XLNetTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLNetTokenizer, TFXLNetModel
import tensorflow as tf

# xlnet-base-cased
MODEL_NAME = 'xlnet-base-cased'

XLNetTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFXLNetModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFXLNetModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    output = model(input)
    return {"last_hidden_state": output.last_hidden_state}

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

try downloading TF weights


tf_model.h5:   0%|          | 0.00/565M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical value

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 456116
-rw-r--r-- 1 root root       882 May 15 23:04 config.json
drwxr-xr-x 3 root root      4096 May 15 23:04 saved_model
-rw-r--r-- 1 root root 467052976 May 15 23:05 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 6616
drwxr-xr-x 2 root root    4096 May 15 23:04 assets
-rw-r--r-- 1 root root      54 May 15 23:04 fingerprint.pb
-rw-r--r-- 1 root root   84867 May 15 23:04 keras_metadata.pb
-rw-r--r-- 1 root root 6675455 May 15 23:04 saved_model.pb
drwxr-xr-x 2 root root    4096 May 15 23:04 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 788
-rw-r--r-- 1 root root    346 May 15 23:03 special_tokens_map.json
-rw-r--r-- 1 root root 798011 May 15 23:03 spiece.model
-rw-r--r-- 1 root root   2025 May 15 23:03 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for

In [6]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {MODEL_NAME}/saved_model/1/assets

## Import and Save XLNet in Spark NLP


In [7]:
!zip -r test_xlnet.zip /content/xlnet-base-cased/saved_model/1

  adding: content/xlnet-base-cased/saved_model/1/ (stored 0%)
  adding: content/xlnet-base-cased/saved_model/1/variables/ (stored 0%)
  adding: content/xlnet-base-cased/saved_model/1/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/xlnet-base-cased/saved_model/1/variables/variables.index (deflated 76%)
  adding: content/xlnet-base-cased/saved_model/1/saved_model.pb (deflated 91%)
  adding: content/xlnet-base-cased/saved_model/1/assets/ (stored 0%)
  adding: content/xlnet-base-cased/saved_model/1/assets/spiece.model (deflated 49%)
  adding: content/xlnet-base-cased/saved_model/1/fingerprint.pb (stored 0%)
  adding: content/xlnet-base-cased/saved_model/1/keras_metadata.pb (deflated 95%)


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlnetEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlnetEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively..


In [ ]:
from sparknlp.annotator import *

xlnet = XlnetEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('xlnet_base_cased')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
xlnet.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XLNet model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊

In [ ]:
xlnet_loaded = XlnetEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [ ]:
xlnet_loaded.getStorageRef()

That's it! You can now go wild and use hundreds of XLNet models from HuggingFace 🤗 in Spark NLP 🚀
